In [1]:
import numpy as np
import pandas as pd

In [2]:
harp_data = pd.read_parquet("../harp_data/data/processed/aggregated_high-qual_near-center-70.parquet")
flux_data = pd.read_parquet("../xray_fluxes/data/processed/1m_data.parquet")
flare_data = pd.read_csv("../flare_data/sci_20100101_20240721.csv")

In [3]:
print(harp_data.shape)
print(flux_data.shape)
print(flare_data.shape)

(571216, 23)
(13847040, 10)
(31281, 16)


In [4]:
flare_data.head()

,start time,end time,peak time,fl_class,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius
0,2010/1/1 6:02,2010/1/1 6:13,2010/1/1 6:09,B1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010/1/1 12:00,2010/1/1 12:19,2010/1/1 12:09,B2.7,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010/1/1 12:27,2010/1/1 13:09,2010/1/1 12:43,B3.3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010/1/1 15:58,2010/1/1 16:31,2010/1/1 16:20,B2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010/1/1 18:20,2010/1/1 18:31,2010/1/1 18:27,B1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
flare_data.dtypes

start time        object
end time          object
peak time         object
fl_class          object
noaa_ar_5min     float64
noaa_ar_5s       float64
hg1              float64
hg2              float64
car1             float64
car2             float64
rtheta1          float64
rtheta2          float64
xy1              float64
xy2              float64
solar_p_angle    float64
solar_radius     float64
dtype: object

In [6]:
flare_data["start time"] = pd.to_datetime(flare_data["start time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)
flare_data["end time"] = pd.to_datetime(flare_data["end time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)
flare_data["peak time"] = pd.to_datetime(flare_data["peak time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)

In [7]:
flare_data[flare_data["start time"].isna() | flare_data["end time"].isna() | flare_data["peak time"].isna()]

,start time,end time,peak time,fl_class,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius
23,2010-01-03 10:45:00+00:00,NaT,2010-01-03 10:54:00+00:00,B1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2010-01-03 18:52:00+00:00,NaT,2010-01-03 19:02:00+00:00,B1.4,11039.0,11039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,2010-01-03 19:05:00+00:00,NaT,2010-01-03 19:13:00+00:00,B1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2010-01-05 08:40:00+00:00,NaT,2010-01-05 08:44:00+00:00,B1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2010-01-07 02:18:00+00:00,NaT,2010-01-07 02:28:00+00:00,B1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31254,2024-07-18 10:04:00+00:00,NaT,2024-07-18 10:14:00+00:00,M2.2,13751.0,13751.0,337.69840,-7.372751,92.545720,-7.372751,0.429221,118.303474,-356.70746,-192.09494,5.315461,943.90466
31258,2024-07-18 15:07:00+00:00,NaT,2024-07-18 15:18:00+00:00,C4.4,13751.0,13751.0,339.74377,-7.828217,91.797714,-7.828217,0.404812,121.695670,-325.11697,-200.76228,5.407377,943.91800
31269,2024-07-21 12:00:00+00:00,NaT,2024-07-21 12:08:00+00:00,C4.8,0.0,0.0,-9999.00000,-9999.000000,-9999.000000,-9999.000000,1.027328,264.574740,965.56920,-91.70256,6.644676,944.11334
31274,2024-07-21 15:49:00+00:00,NaT,2024-07-21 15:55:00+00:00,C3.3,13744.0,13744.0,-9999.00000,-9999.000000,-9999.000000,-9999.000000,1.054493,286.479000,954.67960,282.40726,6.712200,944.12480


In [8]:
pattern = r"^[A-Z]{1}[1-9][0-9]*\.[0-9]+$"
matches_pattern = flare_data["fl_class"].str.match(pattern)
flare_data[~matches_pattern]

,start time,end time,peak time,fl_class,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius
333,2010-02-08 17:22:00+00:00,2010-02-08 17:36:00+00:00,2010-02-08 17:26:00+00:00,C0.9,11045.0,11045.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3012,2011-05-02 16:02:00+00:00,2011-05-02 16:23:00+00:00,2011-05-02 16:12:00+00:00,C0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4203,2011-09-19 15:31:00+00:00,2011-09-19 15:48:00+00:00,2011-09-19 15:41:00+00:00,M0.9,11301.0,11301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4303,2011-09-27 13:59:00+00:00,2011-09-27 14:09:00+00:00,2011-09-27 14:05:00+00:00,C0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4911,2011-12-05 23:19:00+00:00,2011-12-05 23:35:00+00:00,2011-12-05 23:25:00+00:00,M0.9,11363.0,11363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,2011-12-07 18:10:00+00:00,NaT,2011-12-07 18:17:00+00:00,C0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5509,2012-02-26 23:25:00+00:00,2012-02-26 23:37:00+00:00,2012-02-26 23:33:00+00:00,C0.9,11422.0,11422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5775,2012-03-28 06:09:00+00:00,2012-03-28 06:34:00+00:00,2012-03-28 06:24:00+00:00,C0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6415,2012-05-31 22:11:00+00:00,2012-05-31 22:19:00+00:00,2012-05-31 22:15:00+00:00,C0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6863,2012-07-08 10:16:00+00:00,2012-07-08 10:41:00+00:00,2012-07-08 10:30:00+00:00,M0.9,11515.0,11515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
def get_peak_intensity(fl_class: str) -> float:
    flare_class = fl_class[0]
    multiplier = float(fl_class[1:])
    powers = {"A": 1e-8, "B": 1e-7, "C": 1e-6, "M": 1e-5, "X": 1e-4}
    return multiplier * powers[flare_class]

flare_data.insert(4, "peak_intensity", flare_data["fl_class"].map(get_peak_intensity))

In [10]:
print((flare_data["peak_intensity"].min(), flare_data["peak_intensity"].max()))

(9e-08, 0.00146)


In [11]:
def get_flare_class(peak_intensity: float) -> str:
    thresholds = [10 ** i for i in range(-4, -9, -1)]
    flare_classes = ["X", "M", "C", "B", "A"]
    for threshold, flare_class in zip(thresholds, flare_classes):
        if peak_intensity >= threshold:
            return flare_class
    return pd.NA

flare_data.insert(4, "flare_class", flare_data["peak_intensity"].map(get_flare_class))

In [12]:
flare_data.head()

,start time,end time,peak time,fl_class,flare_class,peak_intensity,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius
0,2010-01-01 06:02:00+00:00,2010-01-01 06:13:00+00:00,2010-01-01 06:09:00+00:00,B1.1,B,1.100000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-01 12:00:00+00:00,2010-01-01 12:19:00+00:00,2010-01-01 12:09:00+00:00,B2.7,B,2.700000e-07,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-01 12:27:00+00:00,2010-01-01 13:09:00+00:00,2010-01-01 12:43:00+00:00,B3.3,B,3.300000e-07,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-01 15:58:00+00:00,2010-01-01 16:31:00+00:00,2010-01-01 16:20:00+00:00,B2.5,B,2.500000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-01 18:20:00+00:00,2010-01-01 18:31:00+00:00,2010-01-01 18:27:00+00:00,B1.3,B,1.300000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print(harp_data["T_REC"].dtype)
print(flux_data["time"].dtype)

datetime64[ns, UTC]
datetime64[us, UTC]


In [14]:
harp_data.head()

,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,...,MEANPOT,TOTPOT,MEANSHR,SHRGT45,NPIX,SIZE,AREA,NACR,SIZE_ACR,AREA_ACR
0,2010-05-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-05-01 00:12:00+00:00,1.439345e+21,32.159885,137.783195,137.832428,59.383249,0.528793,2.065877e+12,0.001307,-0.000677,...,2969.938683,9.237042e+21,24.980931,7.166919,21385.121136,1908.233367,1124.079598,1636.651525,146.041405,86.298480
2,2010-05-01 00:24:00+00:00,1.488903e+21,32.006564,138.032814,137.932202,58.588763,0.355526,2.096785e+12,0.001291,-0.000588,...,2818.961328,9.199543e+21,24.549219,6.112795,22631.582458,2019.510773,1188.056979,1754.132156,156.528544,92.311965
3,2010-05-01 00:36:00+00:00,1.495132e+21,32.489844,136.413847,136.048735,59.049505,0.317466,2.190448e+12,-0.001002,-0.001128,...,2872.564258,9.535298e+21,25.103040,7.290995,23156.134658,2066.389478,1214.339188,1786.976723,159.464861,93.905701
4,2010-05-01 00:48:00+00:00,1.566065e+21,31.843583,136.656334,136.326496,58.237536,0.342064,2.305668e+12,-0.000914,-0.000872,...,2840.389563,9.851970e+21,24.838363,6.565928,24186.736691,2158.422429,1267.802766,1847.651398,164.884269,97.054212


In [15]:
flux_data.head()

,time,satellite_secondary,satellite_primary,flux_secondary,flux_primary,good_data_secondary,good_data_primary,satellite,flux,good_data
0,1998-07-27 00:00:00+00:00,10,None,3.042857e-07,NaN,True,None,10,3.042857e-07,True
1,1998-07-27 00:01:00+00:00,10,None,3.000000e-07,NaN,True,None,10,3.000000e-07,True
2,1998-07-27 00:02:00+00:00,10,None,2.971429e-07,NaN,True,None,10,2.971429e-07,True
3,1998-07-27 00:03:00+00:00,10,None,2.985715e-07,NaN,True,None,10,2.985715e-07,True
4,1998-07-27 00:04:00+00:00,10,None,2.971429e-07,NaN,True,None,10,2.971429e-07,True


In [33]:
t = harp_data['T_REC'][0]
print((t, t + pd.Timedelta(hours=24)))

(Timestamp('2010-05-01 00:00:00+0000', tz='UTC'), Timestamp('2010-05-02 00:00:00+0000', tz='UTC'))


In [34]:
max_fluxes = []
max_flare_classes = []
max_peak_intensities = []

window_start = t
window_end = t + pd.Timedelta(hours=23, minutes=59)

is_in_window = (flux_data["time"] >= window_start) & (flux_data["time"] <= window_end)
max_flux = flux_data.loc[is_in_window, "flux"].max()
max_fluxes.append(max_flux)

is_in_window = (flare_data["peak time"] >= window_start) & (flare_data["peak time"] <= window_end)
if is_in_window.any():
    flare_classes = flare_data.loc[is_in_window, "flare_class"]
    peak_intensities = flare_data.loc[is_in_window, "peak_intensity"]
    i = peak_intensities.idxmax()
    max_flare_class = flare_classes[i]
    max_peak_intensity = peak_intensities[i]
else:
    max_flare_class = pd.NA
    max_peak_intensity = np.nan
max_flare_classes.append(max_flare_class)
max_peak_intensities.append(max_peak_intensity)

print(max_fluxes)
print(max_flare_classes)
print(max_peak_intensities)

[8.111459464998916e-06]
['C']
[8.099999999999999e-06]


In [18]:
combined_data = pd.merge(harp_data, flux_data[["time", "flux"]], how="inner", left_on="T_REC", right_on="time").drop(columns="time")
should_keep = combined_data["T_REC"] + pd.Timedelta(hours=23, minutes=59) <= flux_data["time"].iloc[-1]
combined_data = combined_data[should_keep]
combined_data

,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,...,TOTPOT,MEANSHR,SHRGT45,NPIX,SIZE,AREA,NACR,SIZE_ACR,AREA_ACR,flux
0,2010-05-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.615019e-08
1,2010-05-01 00:12:00+00:00,1.439345e+21,32.159885,137.783195,137.832428,59.383249,0.528793,2.065877e+12,0.001307,-0.000677,...,9.237042e+21,24.980931,7.166919,21385.121136,1908.233367,1124.079598,1636.651525,146.041405,86.298480,6.331756e-08
2,2010-05-01 00:24:00+00:00,1.488903e+21,32.006564,138.032814,137.932202,58.588763,0.355526,2.096785e+12,0.001291,-0.000588,...,9.199543e+21,24.549219,6.112795,22631.582458,2019.510773,1188.056979,1754.132156,156.528544,92.311965,6.583243e-08
3,2010-05-01 00:36:00+00:00,1.495132e+21,32.489844,136.413847,136.048735,59.049505,0.317466,2.190448e+12,-0.001002,-0.001128,...,9.535298e+21,25.103040,7.290995,23156.134658,2066.389478,1214.339188,1786.976723,159.464861,93.905701,7.199385e-08
4,2010-05-01 00:48:00+00:00,1.566065e+21,31.843583,136.656334,136.326496,58.237536,0.342064,2.305668e+12,-0.000914,-0.000872,...,9.851970e+21,24.838363,6.565928,24186.736691,2158.422429,1267.802766,1847.651398,164.884269,97.054212,6.824671e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571211,2024-08-21 23:00:00+00:00,5.555699e+22,44.493295,91.386968,94.154896,49.025126,-0.016395,7.615394e+13,0.012944,0.006413,...,1.227597e+24,38.092000,33.078717,311849.108684,28007.656779,17542.208126,41837.122195,3757.457601,2337.351041,4.116458e-06
571212,2024-08-21 23:12:00+00:00,5.544437e+22,44.508059,91.128024,93.791001,48.946077,-0.005535,7.609429e+13,0.012937,0.006417,...,1.226458e+24,38.122575,33.168761,312174.184838,28037.470321,17540.792603,41922.444216,3765.203258,2339.563178,3.103156e-06
571213,2024-08-21 23:24:00+00:00,5.539495e+22,44.507347,91.014640,93.646220,48.916111,-0.031197,7.632120e+13,0.012649,0.006323,...,1.227736e+24,38.082282,33.030732,312734.513848,28088.429327,17559.179701,41981.686953,3770.609223,2339.675770,2.699496e-06
571214,2024-08-21 23:36:00+00:00,5.547667e+22,44.474201,91.104216,93.844954,48.798387,-0.004176,7.698746e+13,0.012982,0.006429,...,1.231906e+24,38.154950,33.173974,313043.922425,28116.981354,17561.064916,42009.908578,3773.246331,2341.689287,2.528340e-06


In [ ]:
max_fluxes = []
max_flare_classes = []
max_peak_intensities = []
for t in combined_data["T_REC"].iloc[:100]:
    window_start = t
    window_end = t + pd.Timedelta(hours=23, minutes=59)

    is_in_window = (flux_data["time"] >= window_start) & (flux_data["time"] <= window_end)
    max_flux = flux_data.loc[is_in_window, "flux"].max()
    max_fluxes.append(max_flux)

    is_in_window = (flare_data["peak time"] >= window_start) & (flare_data["peak time"] <= window_end)
    flare_classes = flare_data.loc[is_in_window, "flare_class"]
    peak_intensities = flare_data.loc[is_in_window, "peak_intensity"]
    i = peak_intensities.idxmax()
    max_flare_classes.append(flare_classes[i])
    max_peak_intensities.append(peak_intensities[i])
max_fluxes.extend([np.nan for _ in range(len(combined_data) - len(max_fluxes))])
max_flare_classes.extend([np.nan for _ in range(len(combined_data) - len(max_flare_classes))])
max_peak_intensities.extend([np.nan for _ in range(len(combined_data) - len(max_peak_intensities))])

combined_data["max_flux_next_24h"] = max_fluxes
combined_data["max_flare_class_next_24h"] = max_flare_classes
combined_data["max_peak_intensity_next_24h"] = max_peak_intensities

In [20]:
combined_data.head()

,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,...,NPIX,SIZE,AREA,NACR,SIZE_ACR,AREA_ACR,flux,max_flux_next_24h,max_flare_class_next_24h,max_peak_intensity_next_24h
0,2010-05-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.615019e-08,0.000008,C,0.000008
1,2010-05-01 00:12:00+00:00,1.439345e+21,32.159885,137.783195,137.832428,59.383249,0.528793,2.065877e+12,0.001307,-0.000677,...,21385.121136,1908.233367,1124.079598,1636.651525,146.041405,86.298480,6.331756e-08,0.000008,C,0.000008
2,2010-05-01 00:24:00+00:00,1.488903e+21,32.006564,138.032814,137.932202,58.588763,0.355526,2.096785e+12,0.001291,-0.000588,...,22631.582458,2019.510773,1188.056979,1754.132156,156.528544,92.311965,6.583243e-08,0.000008,C,0.000008
3,2010-05-01 00:36:00+00:00,1.495132e+21,32.489844,136.413847,136.048735,59.049505,0.317466,2.190448e+12,-0.001002,-0.001128,...,23156.134658,2066.389478,1214.339188,1786.976723,159.464861,93.905701,7.199385e-08,0.000008,C,0.000008
4,2010-05-01 00:48:00+00:00,1.566065e+21,31.843583,136.656334,136.326496,58.237536,0.342064,2.305668e+12,-0.000914,-0.000872,...,24186.736691,2158.422429,1267.802766,1847.651398,164.884269,97.054212,6.824671e-08,0.000008,C,0.000008


In [56]:
combined_data.to_parquet("../combined_data.parquet")

In [35]:
test = pd.read_parquet("../combined_data/combined_data.parquet")
test

,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,...,NPIX,SIZE,AREA,NACR,SIZE_ACR,AREA_ACR,flux,max_flux_next_24h,max_flare_class_next_24h,max_peak_intensity_next_24h
0,2010-05-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.615019e-08,8.111459e-06,C,8.100000e-06
1,2010-05-01 00:12:00+00:00,1.439345e+21,32.159885,137.783195,137.832428,59.383249,0.528793,2.065877e+12,0.001307,-0.000677,...,21385.121136,1908.233367,1124.079598,1636.651525,146.041405,86.298480,6.331756e-08,8.111459e-06,C,8.100000e-06
2,2010-05-01 00:24:00+00:00,1.488903e+21,32.006564,138.032814,137.932202,58.588763,0.355526,2.096785e+12,0.001291,-0.000588,...,22631.582458,2019.510773,1188.056979,1754.132156,156.528544,92.311965,6.583243e-08,8.111459e-06,C,8.100000e-06
3,2010-05-01 00:36:00+00:00,1.495132e+21,32.489844,136.413847,136.048735,59.049505,0.317466,2.190448e+12,-0.001002,-0.001128,...,23156.134658,2066.389478,1214.339188,1786.976723,159.464861,93.905701,7.199385e-08,8.111459e-06,C,8.100000e-06
4,2010-05-01 00:48:00+00:00,1.566065e+21,31.843583,136.656334,136.326496,58.237536,0.342064,2.305668e+12,-0.000914,-0.000872,...,24186.736691,2158.422429,1267.802766,1847.651398,164.884269,97.054212,6.824671e-08,8.111459e-06,C,8.100000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2010-05-09 07:00:00+00:00,3.692535e+21,29.266211,110.437943,110.821704,40.843044,0.052963,4.910250e+12,0.004833,0.000793,...,46866.607589,4201.053038,2645.208663,3432.073844,307.646003,192.655931,4.949603e-08,2.784999e-07,B,2.700000e-07
996,2010-05-09 07:12:00+00:00,3.649283e+21,29.655435,111.767649,111.894954,41.478713,-0.034500,4.903912e+12,0.004109,0.000572,...,46542.288551,4171.968885,2632.018489,3406.413717,305.344925,191.686674,5.177618e-08,2.784999e-07,B,2.700000e-07
997,2010-05-09 07:24:00+00:00,3.587046e+21,29.650872,112.785731,112.969091,42.133667,0.082730,4.883449e+12,0.004350,0.000734,...,46484.482033,4166.769402,2629.013981,3431.346018,307.578508,193.299612,4.949603e-08,2.784999e-07,B,2.700000e-07
998,2010-05-09 07:36:00+00:00,3.444526e+21,29.510952,114.139043,114.438184,42.933041,0.110068,4.668407e+12,0.005584,0.000815,...,46309.406198,4151.069844,2617.631413,3460.475029,310.189102,194.746323,5.152469e-08,2.784999e-07,B,2.700000e-07


In [36]:
test_1000 = pd.read_parquet("../combined_data/combined_data_1000.parquet")
test_1000

,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,...,NPIX,SIZE,AREA,NACR,SIZE_ACR,AREA_ACR,flux,max_flux_next_24h,max_flare_class_next_24h,max_peak_intensity_next_24h
0,2010-05-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.615019e-08,8.111459e-06,C,8.100000e-06
1,2010-05-01 00:12:00+00:00,1.439345e+21,32.159885,137.783195,137.832428,59.383249,0.528793,2.065877e+12,0.001307,-0.000677,...,21385.121136,1908.233367,1124.079598,1636.651525,146.041405,86.298480,6.331756e-08,8.111459e-06,C,8.100000e-06
2,2010-05-01 00:24:00+00:00,1.488903e+21,32.006564,138.032814,137.932202,58.588763,0.355526,2.096785e+12,0.001291,-0.000588,...,22631.582458,2019.510773,1188.056979,1754.132156,156.528544,92.311965,6.583243e-08,8.111459e-06,C,8.100000e-06
3,2010-05-01 00:36:00+00:00,1.495132e+21,32.489844,136.413847,136.048735,59.049505,0.317466,2.190448e+12,-0.001002,-0.001128,...,23156.134658,2066.389478,1214.339188,1786.976723,159.464861,93.905701,7.199385e-08,8.111459e-06,C,8.100000e-06
4,2010-05-01 00:48:00+00:00,1.566065e+21,31.843583,136.656334,136.326496,58.237536,0.342064,2.305668e+12,-0.000914,-0.000872,...,24186.736691,2158.422429,1267.802766,1847.651398,164.884269,97.054212,6.824671e-08,8.111459e-06,C,8.100000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2010-05-09 07:00:00+00:00,3.692535e+21,29.266211,110.437943,110.821704,40.843044,0.052963,4.910250e+12,0.004833,0.000793,...,46866.607589,4201.053038,2645.208663,3432.073844,307.646003,192.655931,4.949603e-08,2.784999e-07,B,2.700000e-07
996,2010-05-09 07:12:00+00:00,3.649283e+21,29.655435,111.767649,111.894954,41.478713,-0.034500,4.903912e+12,0.004109,0.000572,...,46542.288551,4171.968885,2632.018489,3406.413717,305.344925,191.686674,5.177618e-08,2.784999e-07,B,2.700000e-07
997,2010-05-09 07:24:00+00:00,3.587046e+21,29.650872,112.785731,112.969091,42.133667,0.082730,4.883449e+12,0.004350,0.000734,...,46484.482033,4166.769402,2629.013981,3431.346018,307.578508,193.299612,4.949603e-08,2.784999e-07,B,2.700000e-07
998,2010-05-09 07:36:00+00:00,3.444526e+21,29.510952,114.139043,114.438184,42.933041,0.110068,4.668407e+12,0.005584,0.000815,...,46309.406198,4151.069844,2617.631413,3460.475029,310.189102,194.746323,5.152469e-08,2.784999e-07,B,2.700000e-07


In [37]:
test.equals(test_1000)

True